In [ ]:
from __future__ import print_function
%matplotlib inline

from sklearn import cluster
import histomicstk as htk

import numpy as np
import scipy as sp

import skimage.io
import skimage.measure
import skimage.color


import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import cv2
import numpy as np
import math

#Some nice default configuration for plots
plt.rcParams['figure.figsize'] = 15, 15
plt.rcParams['image.cmap'] = 'gray'
titlesize = 24

In [ ]:
def circle_fit(x_coords,y_coords,pixel_len):
    # Pre-processing for Circle Fitting Algorithm for Histological objects
    x1 = np.array(x_coords) # Array of x-coordinates 
    y1 = np.array(y_coords) # Array of y-coordinates

    # Apply transformation on x,y co-ordinates
    x2 = x1.reshape(1, pixel_len)
    y2 = y1.reshape(1, pixel_len)

    # Mean value of x co-ordinates
    sum_x=0
    for i in range(pixel_len):
        sum_x=sum_x+x2[0,i]
    x_mean=sum_x/(pixel_len -1)

    # Mean value of y co-ordinates
    sum_y=0
    for i in range(pixel_len):
        sum_y=sum_y+y2[0,i]
    y_mean=sum_y/(pixel_len - 1)
    
    # Circle Fitting Algorithm using linear algebra    
    u=[]
    v=[]
    Suv  = 0
    Suu  = 0
    Svv  = 0
    Suuv = 0
    Suvv = 0
    Suuu = 0
    Svvv = 0

    # calculation of the reduced coordinates
    for i in range(pixel_len):
            u.append((x2[0,i] - x_mean))
            v.append((y2[0,i] - y_mean))

    # linear system defining the center (uc, vc) in reduced coordinates:
    #    Suu * uc +  Suv * vc = (Suuu + Suvv)/2
    #    Suv * uc +  Svv * vc = (Suuv + Svvv)/2
    for i in range(pixel_len):
        Suv  = Suv + (u[i]*v[i])
        Suu  = Suu+(u[i]*u[i])
        Svv  = Svv+(v[i]*v[i])
        Suuv = Suuv+(u[i]*u[i]*v[i])
        Suvv = Suvv+(u[i]* v[i]**2)
        Suuu = Suuu+(u[i]**3)
        Svvv = Svvv+(v[i]**3)

    # Solving the linear system
    A = np.array([ [ Suu, Suv ], [Suv, Svv]])
    B = np.array([ Suuu + Suvv, Svvv + Suuv ])/2.0
    uc, vc = np.linalg.solve(A, B)

    xc_1 = x_mean + uc
    yc_1 = y_mean + vc

    # Calcuating Radius
    alpha = uc**2 + vc**2 +((Suu+Svv)/(pixel_len - 1))
    radius=math.sqrt(alpha)
    
    return xc_1, yc_1,radius 

In [ ]:
inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png

imInput = skimage.io.imread(inputImageFile)[:, :, :3]

plt.imshow(imInput)
_ = plt.title('Input Image', fontsize=16)

In [ ]:
# Converting 3D image data into 2D data for k-means clustering
x, y, z = imInput.shape
image_2d = imInput.reshape(x*y, z)
image_2d.shape

In [ ]:
# Applying K-means clusters on the given image
# As the input is image object, the clusters are color intensity
# Based on pixel (colored) distances from the respective cluster_centers
kmeans_cluster_00 = cluster.KMeans(n_clusters=3)
kmeans_cluster_00.fit(image_2d)
cluster_centers_00 = kmeans_cluster_00.cluster_centers_
cluster_labels_00 = kmeans_cluster_00.labels_

In [ ]:
cluster_labels_00.shape

In [ ]:
# Each row represents respective cluster# 
# Each column in each row represents color intensity (RGB)
# For Eg, in this case, 3 clusters with their RGB values
# Cluster#0 is clustered with (243.01990705, 229.31279562, 233.74852959)
cluster_centers_00

In [ ]:
cluster_labels_00

In [ ]:
cluster_labels1=np.zeros((256,256))

In [ ]:
cluster_labels1= cluster_labels.reshape(256, 256)

In [ ]:
cluster_labels1.shape

In [ ]:
imOutput=cluster_centers_00[cluster_labels_00].reshape(x, y, z)

In [ ]:
plt.figure(figsize = (15,8))
plt.imshow((imOutput * 255).astype(np.uint8))

In [ ]:
imOutput.shape, imInput.shape

In [ ]:
imOutput.size

# Finding Largest Circle Object of Tissue component  (LOT)  - Procedure

# Finding LOT for first cluster (Cluster0 of 1st run of k-means)

In [ ]:
# Adding up three cluster_positions with their x,y coordinates
clust_00_x=[]
clust_00_y=[]
clust_01_x=[]
clust_01_y=[]
clust_02_x=[]
clust_02_y=[]
for i in range(0,255):
       for j in range(0,255):
        if (cluster_labels1[i,j] == 0): clust_00_x.append([i]),clust_00_y.append([j])
        if (cluster_labels1[i,j] == 1): clust_01_x.append([i]),clust_01_y.append([j])
        if (cluster_labels1[i,j] == 2): clust_02_x.append([i]),clust_02_y.append([j])
z=len(clust_00_x)+len(clust_01_x)+len(clust_02_x)
z

In [ ]:
len(clust_00_x),len(clust_01_x),len(clust_02_x)

In [ ]:
# Combining x,y coordinates for cluster0
cluster_00_xy = np.vstack((cluster_00_x, cluster_00_y)).T 

In [ ]:
# Identifying image positions/pixels matching nearest cluster0 intenstity values 
cluster_00_points = np.where(imInput ==(243, 229, 233)) #[243.01990705, 229.31279562, 233.74852959] Cluster#0 values

In [ ]:
# Splitting up image positios/pixels of cluster_00_points into another 3 clusters
# k-means clustering on points distance from cluster_center
# As the input is pixel co-ordinates, clusters are based pixel distance from cluster_center
cluster_00_0x=[]
cluster_00_1x=[]
cluster_00_2x=[]
cluster_00_0y=[]
cluster_00_1y=[]
cluster_00_2y=[]
for i in range(0,3442):
        if (cluster_00_points[2][i] == 0): cluster_00_0x.append(cluster_00_points[0][i]), cluster_00_0y.append(cluster_00_points[1][i])
        if (cluster_00_points[2][i] == 1): cluster_00_1x.append(cluster_00_points[0][i]), cluster_00_1y.append(cluster_00_points[1][i])
        if (cluster_00_points[2][i] == 2): cluster_00_2x.append(cluster_00_points[0][i]), cluster_00_2y.append(cluster_00_points[1][i])  

In [ ]:
# Combining x,y coordinates for first cluster of cluster00
cluster_00_0_xy = np.vstack((cluster_00_0x, cluster_00_0y)).T 

In [ ]:
kmeans_cluster_00_1 = cluster.KMeans(n_clusters=3)
kmeans_cluster_00_1.fit(cluster_00_0_xy)
cluster_00_1_centers= kmeans_cluster_00_1.cluster_centers_
cluster_00_1_labels= kmeans_cluster_00_1.labels_

In [ ]:
cluster_00_1_centers

In [ ]:
# Identifying image positions/pixels matching nearest cluster0 intenstity values 
cluster_00_1_points = np.where(imInput ==()) #[243.01990705, 229.31279562, 233.74852959] Cluster#0 values

In [ ]:
# Splitting up cluster_00_0_points into another 3 clusters 
# First cluster from cluster0(Image)->cluster0(coordinates)
# k-means clustering on points distance from cluster_center
# As the input is pixel co-ordinates, clusters are based pixel distance from cluster_center
cluster_00_0_0x=[]
cluster_00_0_1x=[]
cluster_00_0_2x=[]
cluster_00_0_0y=[]
cluster_00_0_1y=[]
cluster_00_0_2y=[]
for i in range(0,3442):
        if (cluster_00_1_points[2][i] == 0): cluster_00_0_0x.append(cluster_00_1_points[0][i]), cluster_00_0_0y.append(cluster_00_1_points[1][i])
        if (cluster_00_1_points[2][i] == 1): cluster_00_0_1x.append(cluster_00_1_points[0][i]), cluster_00_0_1y.append(cluster_00_1_points[1][i])
        if (cluster_00_1_points[2][i] == 2): cluster_00_0_2x.append(cluster_00_1_points[0][i]), cluster_00_0_2y.append(cluster_00_1_points[1][i])  

In [ ]:
# Combining x,y coordinates for first cluster of cluster00
cluster_00_0_0_xy = np.vstack((cluster_00_0_0x, cluster_00_0_0y)).T 

In [ ]:
! pip install circle-fit
import circle_fit as cf

In [ ]:
# Applying Circle-fit algorithm on each cluster
xc_00,yc_00,r_00,_ = cf.least_squares_circle((cluster_00_xy))
xc_00_0,yc_00_0,r_00_0,_ = cf.least_squares_circle((cluster_00_0_xy))
xc_00_0_0,yc_00_0_0,r_00_0_0,_ = cf.least_squares_circle((cluster_00_0_0_xy))

In [ ]:
# Drawing Circles for each cluster
inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()

cv2.circle(imInput,(int(xc_00),int(yc_00)),int(r_00),3)
cv2.circle(imInput,(int(xc_00_0),int(yc_00_0)),int(r_00_0),3)
cv2.circle(imInput,(int(xc_00_0_0),int(yc_00_0_0)),int(r_00_0_0),3)

cv2.imwrite('/home/raj/Downloads/clustered_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))
           
im = cv2.imread('/home/raj/Downloads/clustered_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')

plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
np.shape(cluster_0_points)r

In [ ]:
# First Row represents, x-coordinates of position, Second row represents y-coordinates
# Third row represents respective cluster Number
# For Eg, the first columun value (0,136,0) represents, (0,136) position in the image with cluster#0
cluster_0_points

In [ ]:
# Cluster#0 intensity values [243.01990705, 229.31279562, 233.74852959]
# cluster_0, cluster_0, cluster_1, cluster_0
# Position's Intensity values closely matches with respective cluster's intensity values
imInput[0,136], imInput[0,138], imInput[255,198], imInput[255,236] 

In [ ]:
# Identifying image positions/pixels matching nearest cluster1 intenstity values 
cluster_1_points = np.where(imInput ==(211, 127, 163)) #[211.21876855, 127.25825318, 163.44356371] Cluster#1 values

In [ ]:
cluster_1_points

In [ ]:
# Cluster#1 intensity values [211.21876855, 127.25825318, 163.44356371]
# cluster_0, cluster_1, cluster_2
# Position's Intensity values closely matches with respective cluster's intensity values
imInput[0,33], imInput[0,56], imInput[255,226] 

In [ ]:
# Identifying image positions/pixels matching nearest cluster2 intenstity values 
cluster_2_points = np.where(imInput ==(142, 71, 121)) #[142.47274174,  71.00358138, 121.60684441] Cluster#2 values

In [ ]:
cluster_2_points

In [ ]:
# Cluster#1 intensity values [142.47274174,  71.00358138, 121.60684441]
# cluster_0, , cluster_1, cluster_2
# Position's Intensity values closely matches with respective cluster's intensity values
imInput[0,19],imInput[1,0], imInput[0,204]

In [ ]:
print(np.shape(cluster_0_points))
print(np.shape(cluster_1_points))
print(np.shape(cluster_2_points))

In [ ]:
cluster_0_points[0].shape, cluster_1_points[0].shape, cluster_2_points[0].shape

In [ ]:
#Splitting up image positios/pixels of cluster_1_points into another 3 clusters based on clusteral values 
cluster_1_0x=[]
cluster_1_1x=[]
cluster_1_2x=[]
cluster_1_0y=[]
cluster_1_1y=[]
cluster_1_2y=[]
for i in range(0,2326):
        if (cluster_1_points[2][i] == 0): cluster_1_0x.append(cluster_1_points[0][i]), cluster_1_0y.append(cluster_1_points[1][i])
        if (cluster_1_points[2][i] == 1): cluster_1_1x.append(cluster_1_points[0][i]), cluster_1_1y.append(cluster_1_points[1][i])
        if (cluster_1_points[2][i] == 2): cluster_1_2x.append(cluster_1_points[0][i]), cluster_1_2y.append(cluster_1_points[1][i])  

In [ ]:
#Splitting up image positios/pixels of cluster_2_points into another 3 clusters based on clusteral values 
cluster_2_0x=[]
cluster_2_1x=[]
cluster_2_2x=[]
cluster_2_0y=[]
cluster_2_1y=[]
cluster_2_2y=[]
for i in range(0,377):
        if (cluster_2_points[2][i] == 0): cluster_2_0x.append(cluster_2_points[0][i]), cluster_2_0y.append(cluster_2_points[1][i])
        if (cluster_2_points[2][i] == 1): cluster_2_1x.append(cluster_2_points[0][i]), cluster_2_1y.append(cluster_2_points[1][i])
        if (cluster_2_points[2][i] == 2): cluster_2_2x.append(cluster_2_points[0][i]), cluster_2_2y.append(cluster_2_points[1][i])  

### start

In [ ]:
# Cluster_0_0 Circle (Largest Object Circle) generation
cluste_0_0_len=len(cluster_0_0x)
cluster_0_0_x, cluster_0_0_y, cluster_0_0_radius = circle_fit(cluster_0_0x, cluster_0_0y, cluste_0_0_len)

inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()
cv2.circle(imInput,(int(round(cluster_0_0_x)),int(round(cluster_0_0_y))),int(round(cluster_0_0_radius)), 3)           
cv2.imwrite('/home/raj/Downloads/cluster_0_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))
           
im = cv2.imread('/home/raj/Downloads/cluster_0_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')

plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Cluster_0_1 Circle (Largest Object Circle) generation
cluste_0_1_len=len(cluster_0_1x)
cluster_0_1_x, cluster_0_1_y, cluster_0_1_radius = circle_fit(cluster_0_1x, cluster_0_1y, cluste_0_1_len)

inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()
cv2.circle(imInput,(int(round(cluster_0_1_x)),int(round(cluster_0_1_y))),int(round(cluster_0_1_radius)), 3)           
cv2.imwrite('/home/raj/Downloads/cluster_0_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))           

im = cv2.imread('/home/raj/Downloads/cluster_0_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Cluster_0_2 Circle (Largest Object Circle) generation
cluste_0_2_len=len(cluster_0_2x)
cluster_0_2_x, cluster_0_2_y, cluster_0_2_radius = circle_fit(cluster_0_2x, cluster_0_2y, cluste_0_2_len)

inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()
cv2.circle(imInput,(int(round(cluster_0_2_x)),int(round(cluster_0_2_y))),int(round(cluster_0_2_radius)), 3)           
cv2.imwrite('/home/raj/Downloads/cluster_0_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))           

im = cv2.imread('/home/raj/Downloads/cluster_0_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Cluster_1_0 Circle (Largest Object Circle) generation
cluste_1_0_len=len(cluster_1_0x)
cluster_1_0_x, cluster_1_0_y, cluster_1_0_radius = circle_fit(cluster_1_0x, cluster_1_0y, cluste_1_0_len)

inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()
cv2.circle(imInput,(int(round(cluster_1_0_x)),int(round(cluster_1_0_y))),int(round(cluster_1_0_radius)), 3)           
cv2.imwrite('/home/raj/Downloads/cluster_1_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))
           
im = cv2.imread('/home/raj/Downloads/cluster_1_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')

plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Cluster_1_1 Circle (Largest Object Circle) generation
cluste_1_1_len=len(cluster_1_1x)
cluster_1_1_x, cluster_1_1_y, cluster_1_1_radius = circle_fit(cluster_1_1x, cluster_1_1y, cluste_1_1_len)

inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()
cv2.circle(imInput,(int(round(cluster_1_1_x)),int(round(cluster_1_1_y))),int(round(cluster_1_1_radius)), 3)           
cv2.imwrite('/home/raj/Downloads/cluster_1_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))
           
im = cv2.imread('/home/raj/Downloads/cluster_1_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')

plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Cluster_1_2 Circle (Largest Object Circle) generation
cluste_1_2_len=len(cluster_1_2x)
cluster_1_2_x, cluster_1_2_y, cluster_1_2_radius = circle_fit(cluster_1_2x, cluster_1_2y, cluste_1_2_len)

inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()
cv2.circle(imInput,(int(round(cluster_1_2_x)),int(round(cluster_1_2_y))),int(round(cluster_1_2_radius)), 3)           
cv2.imwrite('/home/raj/Downloads/cluster_1_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))
           
im = cv2.imread('/home/raj/Downloads/cluster_1_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')

plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Cluster_2_0 Circle (Largest Object Circle) generation
cluste_2_0_len=len(cluster_2_0x)
cluster_2_0_x, cluster_2_0_y, cluster_2_0_radius = circle_fit(cluster_2_0x, cluster_2_0y, cluste_2_0_len)

inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()
cv2.circle(imInput,(int(round(cluster_2_0_x)),int(round(cluster_2_0_y))),int(round(cluster_2_0_radius)), 3)           
cv2.imwrite('/home/raj/Downloads/cluster_2_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))
           
im = cv2.imread('/home/raj/Downloads/cluster_2_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')

plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Cluster_2_1 Circle (Largest Object Circle) generation
cluste_2_1_len=len(cluster_2_1x)
cluster_2_1_x, cluster_2_1_y, cluster_2_1_radius = circle_fit(cluster_2_1x, cluster_2_1y, cluste_2_1_len)

inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()
cv2.circle(imInput,(int(round(cluster_2_1_x)),int(round(cluster_2_1_y))),int(round(cluster_2_1_radius)), 3)           
cv2.imwrite('/home/raj/Downloads/cluster_2_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))
           
im = cv2.imread('/home/raj/Downloads/cluster_2_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')

plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Cluster_2_2 Circle (Largest Object Circle) generation
cluste_2_2_len=len(cluster_2_2x)
cluster_2_2_x, cluster_2_2_y, cluster_2_2_radius = circle_fit(cluster_2_2x, cluster_2_2y, cluste_2_2_len)

inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()
cv2.circle(imInput,(int(round(cluster_2_2_x)),int(round(cluster_2_2_y))),int(round(cluster_2_2_radius)), 3)           
cv2.imwrite('/home/raj/Downloads/cluster_2_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))
           
im = cv2.imread('/home/raj/Downloads/cluster_2_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')

plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,10))

ax1 = fig.add_subplot(3,3,1)
ax1.set_title("Cluster0")
im1 = cv2.imread('/home/raj/Downloads/cluster_0_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,2)
ax1.set_title("Cluster1")
im1 = cv2.imread('/home/raj/Downloads/cluster_0_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,3)
ax1.set_title("Cluster2")
im1 = cv2.imread('/home/raj/Downloads/cluster_0_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,4)
ax1.set_title("Cluster3")
im1 = cv2.imread('/home/raj/Downloads/cluster_1_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,5)
ax1.set_title("Cluster4")
im1 = cv2.imread('/home/raj/Downloads/cluster_1_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,6)
ax1.set_title("Cluster5")
im1 = cv2.imread('/home/raj/Downloads/cluster_1_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,7)
ax1.set_title("Cluster6")
im1 = cv2.imread('/home/raj/Downloads/cluster_2_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,8)
ax1.set_title("Cluster7")
im1 = cv2.imread('/home/raj/Downloads/cluster_2_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,9)
ax1.set_title("Cluster8")
im1 = cv2.imread('/home/raj/Downloads/cluster_2_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))
plt.show()

## Using python package circle-fit

In [ ]:
! pip install circle-fit

In [ ]:
import circle_fit as cf

In [ ]:
# Cluster_0_0 Circle (Largest Object Circle) generation
combined2_2 = np.vstack((cluster_2_2x, cluster_2_2y)).T
xc_22,yc_22,r_22,_ = cf.least_squares_circle((combined2_2))

inputImageFile = ('/media/raj/Raj1_5/10ktiles/train/gbm/TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')  # H&E.png
imOrig = skimage.io.imread(inputImageFile)[:, :, :3]

imInput = imOrig.copy()
cv2.circle(imInput,(int(xc_22),int(yc_22)),int(r_22),3)
#cv2.circle(imInput,(int(round(cluster_0_0_x)),int(round(cluster_0_0_y))),int(round(cluster_0_0_radius)), 3)           
cv2.imwrite('/home/raj/Downloads/cluster_cf_2_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png',
            cv2.cvtColor(imInput, cv2.COLOR_RGB2BGR))
           
im = cv2.imread('/home/raj/Downloads/cluster_cf_2_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')

plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,10))

ax1 = fig.add_subplot(3,3,1)
ax1.set_title("Cluster0")
im1 = cv2.imread('/home/raj/Downloads/cluster_cf_0_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,2)
ax1.set_title("Cluster1")
im1 = cv2.imread('/home/raj/Downloads/cluster_cf_0_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,3)
ax1.set_title("Cluster2")
im1 = cv2.imread('/home/raj/Downloads/cluster_cf_0_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,4)
ax1.set_title("Cluster3")
im1 = cv2.imread('/home/raj/Downloads/cluster_cf_1_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,5)
ax1.set_title("Cluster4")
im1 = cv2.imread('/home/raj/Downloads/cluster_cf_1_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,6)
ax1.set_title("Cluster5")
im1 = cv2.imread('/home/raj/Downloads/cluster_cf_1_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,7)
ax1.set_title("Cluster6")
im1 = cv2.imread('/home/raj/Downloads/cluster_cf_2_0_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,8)
ax1.set_title("Cluster7")
im1 = cv2.imread('/home/raj/Downloads/cluster_cf_2_1_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))

ax1 = fig.add_subplot(3,3,9)
ax1.set_title("Cluster8")
im1 = cv2.imread('/home/raj/Downloads/cluster_cf_2_2_TCGA-02-0001-01Z-00-DX2_20x_4768_25216_256x256.png')
ax1.imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))
plt.show()